In [1]:
import re

import langid
import numpy as np
import pandas as pd

In [2]:
df_filename = "tweets_Lululemon.csv"
df = pd.read_csv(df_filename, index_col='id', low_memory = False)

In [3]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['date'] = df['timestamp'].dt.date

In [4]:
df.head()

,timestamp,content,username,date
id,,,,
1174835548688531456,2019-09-19 23:59:56+00:00,Check out what I just added to my closet on Po...,rhapsody112,2019-09-19
1174834981811572736,2019-09-19 23:57:41+00:00,Check out what I just added to my closet on Po...,rhapsody112,2019-09-19
1174834959170715648,2019-09-19 23:57:36+00:00,@lisarunz @lululemon Lol...if I had time I wou...,keepitrealmom,2019-09-19
1174834162412355584,2019-09-19 23:54:26+00:00,Check out what I just added to my closet on Po...,GoodBadStupid,2019-09-19
1174832820121821186,2019-09-19 23:49:06+00:00,Check out what I just added to my closet on Po...,PiazzaTalia,2019-09-19


In [5]:
#Clean the text and drop tweets that have only tags (probably those with multimedia items)
def clean_content(text):
   list_content = text.split()
   list_content_new = []
   list_content_new[:] = [tup for tup in list_content if (tup[0] != "@" and tup[:4] != "http")]
   temp_phrase = re.sub("[^a-zA-Z0-9 \’\']","",' '.join(list_content_new))
   temp_phrase = temp_phrase.replace("’", "'")
   temp_l = temp_phrase.split()
   temp_phrase = ' '.join(temp_l)
   return temp_phrase

df['clean_content'] = ''
df['clean_content'] = df.apply(lambda row: clean_content(row['content']), axis = 1)
df['clean_content'].replace('', np.nan, inplace=True)
df.dropna(subset=['clean_content'], inplace=True)


In [6]:
def find_lang(content_str):
    lang = langid.classify(content_str)
    return lang[0]
df['lang'] = df.apply(lambda row: find_lang(row['clean_content']), axis = 1)

In [9]:
df[df['lang']!='en'].shape

(124, 6)

In [39]:
df.head()

,timestamp,content,username,date,lang,clean_content
id,,,,,,
1174835548688531456,2019-09-19 23:59:56+00:00,Check out what I just added to my closet on Po...,rhapsody112,2019-09-19,en,Check out what I just added to my closet on Po...
1174834981811572736,2019-09-19 23:57:41+00:00,Check out what I just added to my closet on Po...,rhapsody112,2019-09-19,en,Check out what I just added to my closet on Po...
1174834959170715648,2019-09-19 23:57:36+00:00,@lisarunz @lululemon Lol...if I had time I wou...,keepitrealmom,2019-09-19,en,Lolif I had time I would Social Working is all...
1174834162412355584,2019-09-19 23:54:26+00:00,Check out what I just added to my closet on Po...,GoodBadStupid,2019-09-19,en,Check out what I just added to my closet on Po...
1174832820121821186,2019-09-19 23:49:06+00:00,Check out what I just added to my closet on Po...,PiazzaTalia,2019-09-19,en,Check out what I just added to my closet on Po...


In [40]:
df.groupby('date').nunique()

,timestamp,content,username,lang,clean_content
date,,,,,
2019-09-15,357,355,208,16,343
2019-09-16,347,346,201,11,341
2019-09-17,451,447,256,19,432
2019-09-18,373,371,243,20,362
2019-09-19,392,392,235,13,389


In [20]:
df.groupby('lang').size()

lang
af       4
ca       1
cs       2
cy       3
da       6
de       6
en    1788
es       9
et       9
fi       1
fr      13
it       3
nl       4
no       4
pl       1
pt       1
ro      56
so       5
sv       3
sw       1
tl       7
tr       5
vi       2
dtype: int64

In [10]:
df_eng = df[df['lang']=='en'].copy(deep=True)

In [11]:
df_eng.to_csv('Lululemon_tweets_en.csv')
df.to_csv('tweets_Lululemon_processed.csv')